## Librerias Usadas

In [3]:
pip install memory-profiler

In [4]:
%load_ext memory_profiler

In [1]:
import json
import pandas as pd
import q1_time as q1t
import q1_memory as q1m
import q2_time as q2t
import q2_memory as q2m
import q3_time as q3t
import timeit
import cProfile
import pstats
from io import StringIO
import dataset_treatment as dt

## Carga del Archivo

En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [2]:
file_path = "farmers-protest-tweets-2021-2-4.json"

Cabe resaltar que el archivo json lo incluimos en el .gitignore para no subirlo a git nunca pues es bastante pesado

In [6]:
# Lee el archivo JSON línea por línea y carga cada línea como un objeto JSON
with open(file_path, 'r') as file:
    data = [json.loads(line) for line in file]

# Convierte los datos en un DataFrame
df = pd.DataFrame(data)

## Analisis Exploratorio

Durante esta sección haremos un pequeño analisis exploratorio sobre el DataFrame <em> farmers-protest-tweets-2021-2-4.json </em> para conocer un poco sobre el comportamiento de los datos

In [7]:
# Muestra de la primera fila de DF
df.head(1)

,url,date,content,renderedContent,id,user,outlinks,tcooutlinks,replyCount,retweetCount,...,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,media,retweetedTweet,quotedTweet,mentionedUsers
0,https://twitter.com/ArjunSinghPanam/status/136...,2021-02-24T09:23:35+00:00,The world progresses while the Indian police a...,The world progresses while the Indian police a...,1364506249291784198,"{'username': 'ArjunSinghPanam', 'displayname':...",[https://twitter.com/ravisinghka/status/136415...,[https://t.co/es3kn0IQAF],0,0,...,0,1364506249291784198,en,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,None,None,{'url': 'https://twitter.com/RaviSinghKA/statu...,"[{'username': 'narendramodi', 'displayname': '..."


Haremos un recorte de nuestro DataFrame dado que solo necesitamos algunos de los atributos del DF para realizar nuestro challenge:
1) Date, user, id
2) content
3) user, mentionedUsers

In [8]:
df= df.filter(items=['id','date','content','user','mentionedUsers'])
df.head()

,id,date,content,user,mentionedUsers
0,1364506249291784198,2021-02-24T09:23:35+00:00,The world progresses while the Indian police a...,"{'username': 'ArjunSinghPanam', 'displayname':...","[{'username': 'narendramodi', 'displayname': '..."
1,1364506237451313155,2021-02-24T09:23:32+00:00,#FarmersProtest \n#ModiIgnoringFarmersDeaths \...,"{'username': 'PrdeepNain', 'displayname': 'Pra...","[{'username': 'Kisanektamorcha', 'displayname'..."
2,1364506195453767680,2021-02-24T09:23:22+00:00,ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾਂ ਨੂੰ ਮੱਦੇਨਜ਼ਰ ਰੱਖਦੇ ਹੋਏ \nਮੇ...,"{'username': 'parmarmaninder', 'displayname': ...",None
3,1364506167226032128,2021-02-24T09:23:16+00:00,@ReallySwara @rohini_sgh watch full video here...,"{'username': 'anmoldhaliwal', 'displayname': '...","[{'username': 'ReallySwara', 'displayname': 'S..."
4,1364506144002088963,2021-02-24T09:23:10+00:00,#KisanEktaMorcha #FarmersProtest #NoFarmersNoF...,"{'username': 'KotiaPreet', 'displayname': 'Pre...",None


Hacemos una validacion en la columna <em> user </em> y <em> mentionedUsers </em> para saber si en cada fila tenemos mas de un registro en estas columnas. Me explico, suponemos que un tweet solo puede ser publicado por un usuario y que en el mismo se puede mencionar a varios usuarios.

In [117]:
# Verificar que cada entrada sea una lista y contar los elementos
def count_objects(lst):
    if isinstance(lst, list):
        return len(lst)
    return 0  # Retornar 0 si no es una lista

# Aplicar la función y filtrar filas donde la cuenta es mayor que 2 para la columna 'user'
df['num_users'] = df['user'].apply(count_objects)
df_more_than_two = df[df['num_users'] > 2]

print("Filas con más de 2 objetos en 'user':")
print(df_more_than_two)

# Aplicar la función y filtrar filas donde la cuenta es mayor que 2 para la columna 'mentionedUsers'
df['num_mentioned'] = df['mentionedUsers'].apply(count_objects)
df_more_than_two = df[df['num_mentioned'] > 2]

print("Filas con más de 2 objetos en 'mentionedUsers':")
print(df_more_than_two['num_mentioned'])

Filas con más de 2 objetos en 'user':
Empty DataFrame
Columns: [id, date, content, user, mentionedUsers, num_users]
Index: []
Filas con más de 2 objetos en 'mentionedUsers':
29        3
52        4
56        3
66        3
86        3
         ..
117305    4
117339    9
117345    5
117355    3
117375    5
Name: num_mentioned, Length: 10692, dtype: int64


Habiendo comprobado el supuesto anterior aplanaremos los datos de la siguiente manera:

1) Vamos aplanar la columna user pues solo necesitaremos username y id.
2) Vamos aplanar el campo mentionedUsers y lo cambiaremos por un diccionario clave (id) valor (username)
3) Ademas vamos a convertir el campo Date a tipo de dato DateTime

In [118]:
# Aplanar la columna 'user' para extraer 'username' y 'id'
df_user = pd.json_normalize(df['user'])

# Seleccionar solo las columnas 'username' e 'id'
df_user = df_user[['username', 'id']]

# Renombrar la columna "id" por "username_id" para evitar conflictos con el id del tweet
df_user = df_user.rename(columns= {'id':'id_username'})

# Unir las nuevas columnas al DataFrame original
df = pd.concat([df, df_user], axis=1)

In [9]:
def list_to_dict(lst):
    if lst is None:
        return None
    return {str(item['id']): item['username'] for item in lst if 'id' in item and 'username' in item}

# Aplicar la función a la columna 'mentionedUsers'
df['mentionedUsers_dict'] = df['mentionedUsers'].apply(list_to_dict)

In [121]:
# Eliminar la columna original 'user' y 'mentionedUsers_dict' ya que no son mas necesarios
df = df.drop(columns=['user'])
df = df.drop(columns=['mentionedUsers'])
df = df.drop(columns=['num_users'])
df = df.drop(columns=['num_mentioned'])

In [122]:
# Convertir la columna 'date' a DateTime
df['date'] = pd.to_datetime(df['date'])

In [10]:
df.head()

,id,date,content,user,mentionedUsers,mentionedUsers_dict
0,1364506249291784198,2021-02-24T09:23:35+00:00,The world progresses while the Indian police a...,"{'username': 'ArjunSinghPanam', 'displayname':...","[{'username': 'narendramodi', 'displayname': '...","{'18839785': 'narendramodi', '1850705408': 'De..."
1,1364506237451313155,2021-02-24T09:23:32+00:00,#FarmersProtest \n#ModiIgnoringFarmersDeaths \...,"{'username': 'PrdeepNain', 'displayname': 'Pra...","[{'username': 'Kisanektamorcha', 'displayname'...",{'1338536920066879488': 'Kisanektamorcha'}
2,1364506195453767680,2021-02-24T09:23:22+00:00,ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾਂ ਨੂੰ ਮੱਦੇਨਜ਼ਰ ਰੱਖਦੇ ਹੋਏ \nਮੇ...,"{'username': 'parmarmaninder', 'displayname': ...",None,None
3,1364506167226032128,2021-02-24T09:23:16+00:00,@ReallySwara @rohini_sgh watch full video here...,"{'username': 'anmoldhaliwal', 'displayname': '...","[{'username': 'ReallySwara', 'displayname': 'S...","{'1546101560': 'ReallySwara', '267861878': 'ro..."
4,1364506144002088963,2021-02-24T09:23:10+00:00,#KisanEktaMorcha #FarmersProtest #NoFarmersNoF...,"{'username': 'KotiaPreet', 'displayname': 'Pre...",None,None


In [124]:
def resumen_columnas(df):
    print("Información del DataFrame:\n")
    print(df.info())
    print("\nConteo de Valores Nulos:\n")
    print(df.isnull().sum())

# Aplicar la función
resumen_columnas(df)

Información del DataFrame:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117407 entries, 0 to 117406
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype              
---  ------               --------------   -----              
 0   id                   117407 non-null  int64              
 1   date                 117407 non-null  datetime64[ns, UTC]
 2   content              117407 non-null  object             
 3   username             117407 non-null  object             
 4   id_username          117407 non-null  int64              
 5   mentionedUsers_dict  38034 non-null   object             
dtypes: datetime64[ns, UTC](1), int64(2), object(3)
memory usage: 5.4+ MB
None

Conteo de Valores Nulos:

id                         0
date                       0
content                    0
username                   0
id_username                0
mentionedUsers_dict    79373
dtype: int64


Hemos terminado nuestro Analisis Exploratorio el cual mas que ir enfocado en detectar valores atípicos, evaluar la distribución de las variables, y visualizar las correlaciones entre ellas. Me he enfocado en entender y comprender el comportamiento de los datos y disminuir el dataset para lo necesario en el challenge

## Top 10 Fechas con mas Tweets

### TIEMPO

Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días. 

In [9]:
# Medir tiempo de ejecución con py-spy
execution_time = timeit.timeit(lambda: q1t.q1_time(file_path), number=10)
print(f"Tiempo de ejecución promedio: {execution_time / 10:.5f} segundos")

Tiempo de ejecución promedio: 6.19171 segundos


In [10]:
# Medir tiempo de ejecución con cProfile
pr = cProfile.Profile()
pr.enable()
q1_time = q1t.q1_time(file_path)
pr.disable()

# Generar reporte
s = StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats(pstats.SortKey.TIME)
ps.print_stats()
print(s.getvalue())

         7985152 function calls (7867314 primitive calls) in 9.375 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   117407    3.988    0.000    3.988    0.000 C:\Users\JuanLH.MYMIRATECH\AppData\Local\anaconda3\Lib\json\decoder.py:343(raw_decode)
   117407    0.645    0.000    0.783    0.000 C:\Users\JuanLH.MYMIRATECH\AppData\Local\anaconda3\Lib\site-packages\pandas\io\json\_normalize.py:182(<dictcomp>)
        1    0.628    0.628    5.368    5.368 C:\Users\JuanLH.MYMIRATECH\Documents\Workspace git\latam-challange\src\dataset_treatment.py:7(<listcomp>)
       13    0.483    0.037    0.483    0.037 {pandas._libs.tslibs.vectorized.ints_to_pydatetime}
   117407    0.384    0.000    0.517    0.000 C:\Users\JuanLH.MYMIRATECH\AppData\Local\anaconda3\Lib\site-packages\pandas\io\json\_normalize.py:184(<dictcomp>)
    49772    0.346    0.000    0.346    0.000 {built-in method _codecs.charmap_decode}
5287077/5287042    0.291    0.00

Utilizamos ambas librerias para medir el tiempo de la solucion del top 10 con mas tweets. Utilizando timeit nos da un promedio de tiempo de 6.19 segundos luego de ponderar 10 ejecuciones sobre la misma función. Por otro lado, cProfile nos da un resumen de tiempo sobre cada paso en la ejecucion de funciones de nuestra solución en la cual podemos ver que gran parte del tiempo se consume en la carga y conversion del .json a un df. Asi que nuestra solución en cuestión de tiempo es decente. 

In [6]:
print("El top 10 fechas con mas tweets enfocado a la optimización de tiempo es: \n")
print(q1_time)

El top 10 fechas con mas tweets enfocado a la optimización de tiempo es: 

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


### MEMORIA

En el caso de la optimización de memoria utilizamos <em> value_counts </em> en lugar de groupby, minimiza la creación de copias de datos, y utiliza tipos de datos más eficientes para las operaciones.

In [11]:
q1_memory = q1m.q1_memory(file_path)

print("El top 10 fechas con mas tweets enfocado a la optimización de memoria es: \n")
print(q1_memory)

Filename: C:\Users\JuanLH.MYMIRATECH\Documents\Workspace git\latam-challange\src\q1_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    16    171.6 MiB    171.6 MiB           1   @profile
    17                                         def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    18                                         
    19                                             # convierte el path a un df con solo las columnas necesarias ['id', 'date', 'username', 'user_id']
    20   1117.8 MiB    946.2 MiB           1       df = dt.dataset_q1(file_path)
    21                                         
    22                                             # Contar tweets por fecha
    23   1117.9 MiB      0.1 MiB           1       tweet_counts = df['date'].value_counts().nlargest(10)
    24                                         
    25                                             # Inicializar una lista para almacenar los resultados
    26   111

Volvemos a recalcar que el mayor consumo de memoria lo podemos ver en la primera parte, en la carga del .json y construcción del dataFrame mas no en la solución brindada para devolver el top 10 de tweets

## Top 10 Emojis

### TIEMPO

In [5]:
# Medir tiempo de ejecución con py-spy
execution_time = timeit.timeit(lambda: q2t.q2_time(file_path), number=10)
print(f"Tiempo de ejecución promedio: {execution_time / 10:.5f} segundos")

Tiempo de ejecución promedio: 4.75675 segundos


In [4]:
# Medir tiempo de ejecución con cProfile
pr = cProfile.Profile()
pr.enable()
q2_time = q2t.q2_time(file_path)
pr.disable()

# Generar reporte
s = StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats(pstats.SortKey.TIME)
ps.print_stats()
print(s.getvalue())

         1745764 function calls (1745717 primitive calls) in 7.564 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   117407    4.183    0.000    4.183    0.000 C:\Users\JuanLH.MYMIRATECH\AppData\Local\anaconda3\Lib\json\decoder.py:343(raw_decode)
        1    0.813    0.813    5.851    5.851 C:\Users\JuanLH.MYMIRATECH\Documents\Workspace git\latam-challange\src\dataset_treatment.py:38(<listcomp>)
   117407    0.425    0.000    0.425    0.000 {method 'findall' of 're.Pattern' objects}
    49772    0.370    0.000    0.370    0.000 {built-in method _codecs.charmap_decode}
        1    0.276    0.276    7.563    7.563 C:\Users\JuanLH.MYMIRATECH\Documents\Workspace git\latam-challange\src\q2_time.py:31(q2_time)
   117407    0.177    0.000    4.498    0.000 C:\Users\JuanLH.MYMIRATECH\AppData\Local\anaconda3\Lib\json\decoder.py:332(decode)
        1    0.171    0.171    0.171    0.171 {pandas._libs.lib.dicts_to_array}
       55  

La optimización de este codigo se da de 3 maneras:
1) La expresión regular emoji_pattern se compila una vez fuera de la función extract_emojis, lo que evita la recompilación en cada llamada, mejorando así la eficiencia.
2) se utiliza una list comprehension para extraer emojis directamente y concatenarlos en una sola lista. Esto evita la sobrecarga de aplicar funciones a nivel de DataFrame y facilita la creación de una lista plana de emojis.
3) 'Counter' se aplica directamente a la lista de emojis extraídos, evitando conversiones adicionales o estructuras intermedias.

Por otro lado, al igual que el anterior ejercicio en lo que mas tiempo se gasta es en la carga y conversion de .json al DataFrame.
Con el metodo py-spy se demora en promedio 6.5 segundos
Con el metodo Cprofile se demora 18.4 segundos

In [3]:
df2 = q2t.q2_time(file_path)
print("El top 10 emojis con su conteo enfocado a la optimización de tiempo es: \n")
df2

El top 10 fechas con mas tweets enfocado a la optimización de tiempo es: 



[('🙏', 1916),
 ('❤️', 952),
 ('😂', 627),
 ('🌾', 529),
 ('💚', 493),
 ('👍', 459),
 ('👉', 450),
 ('✊', 437),
 ('🇮🇳', 399),
 ('👇', 387)]

### MEMORIA

In [3]:
q2_memory = q2m.q2_memory(file_path)

print("El top 10 emojis con su conteo enfocado a la optimización de memoria es: \n")
print(q2_memory)

Filename: C:\Users\JuanLH.MYMIRATECH\Documents\Workspace git\latam-challange\src\q2_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    26    120.6 MiB    120.6 MiB           1   @profile
    27                                         def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    28                                             # convierte el path a un df con solo las columnas necesarias ['content']
    29   1171.9 MiB   1051.4 MiB           1       df = dt.dataset_q2(file_path)
    30                                             
    31                                             # Generador para emojis extraídos
    32   1171.9 MiB      0.0 MiB      160326       emoji_gen = (emoji for text in df['content'] for emoji in extract_emojis(text))
    33                                             
    34                                             # Contar emojis directamente desde el generador
    35   1171.9 MiB      0.0 MiB           1       emoji_cou

La optimización de este codigo se da de 3 maneras:
1) Evitar la creación de listas intermedias grandes y usar generadores cuando sea posible.
2) Utilizar itertools.chain para crear iteradores en lugar de listas.
3) Utilizar collections.Counter eficientemente para acumular conteos en lugar de almacenar todos los emojis y luego contarlos.

Nuevamente el mayor consumo de memoria lo podemos ver en la primera parte, en la carga del .json y construcción del dataFrame mas no en la solución brindada para devolver el top 10 de emojis con sus conteos

##  Top 10 histórico de usuarios mencionando

### TIEMPO

In [4]:
# Medir tiempo de ejecución con py-spy
execution_time = timeit.timeit(lambda: q3t.q3_time(file_path), number=10)
print(f"Tiempo de ejecución promedio: {execution_time / 10:.5f} segundos")

Tiempo de ejecución promedio: 9.31101 segundos


In [5]:
# Medir tiempo de ejecución con cProfile
pr = cProfile.Profile()
pr.enable()
q3_time = q3t.q3_time(file_path)
pr.disable()

# Generar reporte
s = StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats(pstats.SortKey.TIME)
ps.print_stats()
print(s.getvalue())

         26965419 function calls (26378241 primitive calls) in 23.189 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   117407    5.547    0.000    5.547    0.000 C:\Users\JuanLH.MYMIRATECH\AppData\Local\anaconda3\Lib\json\decoder.py:343(raw_decode)
        1    1.237    1.237    8.089    8.089 C:\Users\JuanLH.MYMIRATECH\Documents\Workspace git\latam-challange\src\dataset_treatment.py:57(<listcomp>)
9843903/9843897    0.968    0.000    1.288    0.000 {built-in method builtins.isinstance}
   117522    0.891    0.000    1.313    0.000 {pandas._libs.lib.maybe_convert_objects}
   117407    0.852    0.000    1.047    0.000 C:\Users\JuanLH.MYMIRATECH\AppData\Local\anaconda3\Lib\site-packages\pandas\io\json\_normalize.py:182(<dictcomp>)
   117408    0.851    0.000    7.207    0.000 C:\Users\JuanLH.MYMIRATECH\AppData\Local\anaconda3\Lib\site-packages\pandas\core\series.py:371(__init__)
        1    0.589    0.589   23.189   23.189

In [6]:
print("El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones enfocado a la optimización de tiempo es: \n")
q3_time

El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones enfocado a la optimización de tiempo es: 



[('loyal90901246', 2450),
 ('_gurviir', 1807),
 ('neetuanjle_nitu', 1728),
 ('JaiHind58236291', 1695),
 ('preetysaini321', 926),
 ('JoyJ69957841', 911),
 ('Gurpreetd86', 839),
 ('jot__b', 830),
 ('mani262002', 777),
 ('ScitaramSays', 762)]

La optimización de tiempo sobre este codigo se da de la siguiente forma:

1) Se Utiliza defaultdict para contar de manera eficiente y Counter para una rápida identificación de los elementos más comunes.
2) Se procesa solo las filas relevantes evitando operaciones innecesarias.
3) Se utiliza iterrows para iterar sobre las filas, que es una buena opción cuando la operación por fila es lo suficientemente compleja para amortiguar la sobrecarga de iteración.